In [21]:
import os.path as osp
cfg_path = '/opt/ml/code/my_code/my_custom.py'
wrk_path = osp.join(osp.dirname(cfg_path), f'work_dirs')
wrk_path = osp.join(wrk_path, osp.splitext(osp.basename(cfg_path))[0])
pth_path = osp.join(wrk_path, f'best_bbox_mAP_50.pth')
pkl_path = osp.join(wrk_path, f'inferenced.pkl')
csv_path = osp.join(wrk_path, f'inferenced.csv')

### config 확인

python (print_config.py 경로) (config파일 경로)

In [22]:
!python \
    /opt/ml/code/mmdetection_trash/tools/misc/print_config.py \
    {cfg_path}

Config:
model = dict(
    type='CascadeRCNN',
    pretrained='open-mmlab://regnetx_12gf',
    backbone=dict(
        type='RegNet',
        arch='regnetx_12gf',
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch'),
    neck=[
        dict(
            type='FPN_CARAFE',
            in_channels=[224, 448, 896, 2240],
            out_channels=256,
            num_outs=5,
            start_level=0,
            end_level=-1,
            norm_cfg=None,
            act_cfg=None,
            order=('conv', 'norm', 'act'),
            upsample_cfg=dict(
                type='carafe',
                up_kernel=5,
                up_group=1,
                encoder_kernel=3,
                encoder_dilation=1,
                compressed_channels=64)),
        dict(
            type='BFP',
            in_channels=256,
            num_levels=5,
            refine_level=2,
            refine

### 학습

python (train.py 경로) (config파일 경로)

In [18]:
!python \
    /opt/ml/code/mmdetection_trash/tools/train.py \
    {cfg_path}

 warmup='linear',
    warmup_iters=500,
    warmup_ratio=0.001,
    step=[8, 11])
runner = dict(type='EpochBasedRunner', max_epochs=15)
checkpoint_config = dict(interval=1)
log_config = dict(
    interval=50,
    hooks=[
        dict(type='TextLoggerHook'),
        dict(type='WandbLoggerHook', init_kwargs=dict(project='my_wandb'))
    ])
custom_hooks = [dict(type='NumClassCheckHook')]
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = None
resume_from = None
workflow = [('train', 1)]
classes = ('UNKNOWN', 'General trash', 'Paper', 'Paper pack', 'Metal', 'Glass',
           'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing')
work_dir = './work_dirs/my_custom'
gpu_ids = range(0, 1)

2021-05-17 17:59:06,988 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-17 17:59:06,988 - mmdet - INFO - Use load_from_torchvision loader
2021-05-17 17:59:07,188 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for conv1.weight: 

### 추론

python (test.py 경로) (config파일 경로) (pth파일 경로) --out (저장될 pkl파일 경로) --eval (수행할 작업 목록)


In [48]:
!python \
    /opt/ml/code/mmdetection_trash/tools/test.py \
    {cfg_path} \
    {lst_pth_path} \
    --out {pkl_path} \
    --eval bbox

/opt/conda/lib/python3.7/site-packages/mmcv/utils/registry.py:250: UserWarning: The old API of register_module(module, force=False) is deprecated and will be removed, please use the new API register_module(name=None, force=False, module=None) instead.
  'The old API of register_module(module, force=False) '
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Use load_from_local loader
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 33.5 task/s, elapsed: 25s, ETA:     0s
writing results to /opt/ml/code/my_code/work_dirs/my_faster_rcnn_efficientnet-b0_coco2/inferenced.pkl

Evaluating bbox...
Loading and preparing results...
DONE (t=0.15s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=1.01s).
Accumulating evaluation results...
DONE (t=0.25s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.000
Average 

### 제출 파일 생성

python (pkl_to_submission.py 경로) --pkl (pkl파일 경로) --csv (저장될 csv파일 경로)

In [35]:
!python \
    /opt/ml/code/mmdetection_trash/pkl_to_submission.py \
    --pkl {pkl_path} \
    --csv {csv_path}

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
                                    PredictionString              image_id
0  1 0.23588711 3.6701775 211.15454 82.302536 310...  batch_01_vt/0021.jpg
1  1 0.38276616 0.0 260.0829 273.33008 453.83264 ...  batch_01_vt/0028.jpg
2  1 0.41930428 131.20694 159.13406 381.40994 331...  batch_01_vt/0031.jpg
3  1 0.2507938 384.6004 116.930504 482.76166 248....  batch_01_vt/0032.jpg
4  1 0.2958843 61.64804 458.27533 168.84647 508.5...  batch_01_vt/0070.jpg
